# Working with Spotify tracks
Getting information about Spotify tracks and running basic operations with tracks.

## Import statements

In [ ]:
from my_spotify import get_spotify_object, get_all_tracks_from_playlist
import pandas as pd

## Get the Spotify object

In [ ]:
spot = get_spotify_object('env/.env')

## Specify a playlist to work with

In [ ]:
THE_BEATLES_PLAYLIST_URL_1 = 'https://open.spotify.com/playlist/6ZmqDRJKJf3v3LzYZAaGGU?si=6b487ba5f9b54374' # 215 songs

## Get the tracks from the playlist

In [ ]:
tracks = get_all_tracks_from_playlist(THE_BEATLES_PLAYLIST_URL_1, 'env/.env')

## Check what some items from the "encompassing" dictionary corresponding to a track are
This "encompassing" dictionary includes an "embedded" dictionary pertaining to the track itself, as well as some other metadata.

In [ ]:
# type(tracks)                          # list

t = tracks[0]
# type(track)                           # dict

display(t.keys())
display(t['added_at'])
display(t['is_local'])
display(t['primary_color'])
display(t['video_thumbnail'])

display(t['track'])

## Check what some items from the "embedded" dictionary corresponding to a track are

In [ ]:
track = t['track']
# type(track)                           # dict

display(track.keys())

display(track['track'])
display(track['name'])
display(track['popularity'])
display(track['album'])
display(track['album']['name'])
display(track['duration_ms'])
display(track['artists'][0]['name'])
display(track['id'])
display(track['uri'])
display(track['href'])
display(track['episode'])               # False
display(track['track'])                 # True

## Get the audio features for a track
See [this](https://developer.spotify.com/documentation/web-api/reference/#/operations/get-several-audio-features) for an explanation of the meanings of all these parameters.

In [ ]:
spot.audio_features(['spotify:track:3KiexfmhxHvG5IgAElmTkd'])

## Create a list of URIs of all tracks from the playlist

In [ ]:
uri_list = [t['track']['uri'] for t in tracks]
uri_list[:5]

## Create a list of *(uri, title, popularity, duration)* tuples of all tracks from the playlist

In [ ]:
tracks_data = [(t['track']['uri'],
                t['track']['name'].split(' - Remastered')[0].split(' / Remastered')[0],
                t['track']['popularity'],
                int(round(t['track']['duration_ms'] / 1000, 0))) for t in tracks]
# display(len(tracks_data))
# display(tracks_data[:])

## Create a list of audio features dictionaries of all tracks from the playlist
See [this](https://developer.spotify.com/documentation/web-api/reference/#/operations/get-several-audio-features) for an explanation of the meanings of all these parameters.

**Important:** `spot.audio_features()` will accept max 100 URIs!

In [ ]:
offset = 0
runs, last_run = divmod(len(tracks), 100)                                                       # how many full runs, 100 tracks each
tracks_audio_features_dicts = []

if runs > 0:                                                                                    # all full runs, 100 tracks each
    for _ in range(runs):
        tracks_audio_features_dicts.extend(spot.audio_features(uri_list[offset:(offset+100)]))
        offset += 100
tracks_audio_features_dicts.extend(spot.audio_features(uri_list[offset:(offset+last_run)]))     # last run, generally < 100 tracks

# display(len(tracks_audio_features_dicts))
# display(tracks_audio_features_dicts)

## Create a list of *(key, mode, tempo, time_signature, valence, danceability, energy, loudness, acousticness, instrumentalness, liveness, speechiness)* tuples of all tracks from the playlist
See [this](https://developer.spotify.com/documentation/web-api/reference/#/operations/get-several-audio-features) for an explanation of the meanings of all these parameters.

Use `tracks_audio_features_dicts` list created above.

In [ ]:
tracks_audio_features = [(t['key'],
                          t['mode'],
                          t['tempo'],
                          t['time_signature'],
                          t['valence'],
                          t['danceability'],
                          t['energy'],
                          t['loudness'],
                          t['acousticness'],
                          t['instrumentalness'],
                          t['liveness'],
                          t['speechiness']) for t in tracks_audio_features_dicts]

# display(len(tracks_audio_features))
# display(tracks_audio_features)

## Join the tuples from *tracks_data* and *tracks_audio_features* pairwise to create *tracks_data_and_audio_features*

In [ ]:
tracks_data_and_audio_features = [(d + af) for d, af in zip(tracks_data, tracks_audio_features)]
# display(tracks_data_and_audio_features[:5])

## Bundle it all together in the *get_tracks_data()*, *get_tracks_audio_features()* and *get_tracks_df()* functions
Assumption: the *.env* file is already created as explained in *spotify_authentication.ipynb*, and its relative path is passed as an argument.

### *get_tracks_data()*

In [ ]:
def get_tracks_data(playlist_id: str, env_file_path: str) -> list:
    tracks = get_all_tracks_from_playlist(playlist_id, env_file_path)
    tracks_data = [(t['track']['uri'],
                    t['track']['name'].split(' - Remastered')[0].split(' / Remastered')[0],
                    t['track']['popularity'],
                    int(round(t['track']['duration_ms'] / 1000, 0))) for t in tracks]
    return tracks_data

In [ ]:
# Test get_tracks(data)
display(get_tracks_data(THE_BEATLES_PLAYLIST_URL_1, 'env/.env'))

### *get_tracks_audio_features()*

In [ ]:
def get_tracks_audio_features(playlist_id: str, env_file_path: str) -> list:
    spot = get_spotify_object('env/.env')
    tracks = get_all_tracks_from_playlist(playlist_id, env_file_path)
    uri_list = [t['track']['uri'] for t in tracks]

    offset = 0
    runs, last_run = divmod(len(tracks), 100)                                                       # how many full runs, 100 tracks each
    tracks_audio_features_dicts = []

    if runs > 0:                                                                                    # all full runs, 100 tracks each
        for _ in range(runs):
            tracks_audio_features_dicts.extend(spot.audio_features(uri_list[offset:(offset+100)]))
            offset += 100
    tracks_audio_features_dicts.extend(spot.audio_features(uri_list[offset:(offset+last_run)]))     # last run, < 100 tracks

    tracks_audio_features = [(t['key'],
                              t['mode'],
                              t['tempo'],
                              t['time_signature'],
                              t['valence'],
                              t['danceability'],
                              t['energy'],
                              t['loudness'],
                              t['acousticness'],
                              t['instrumentalness'],
                              t['liveness'],
                              t['speechiness']) for t in tracks_audio_features_dicts]

    return tracks_audio_features

In [ ]:
# Test get_tracks_audio_features()
display(get_tracks_audio_features(THE_BEATLES_PLAYLIST_URL_1, 'env/.env'))

### *get_tracks_df()*

In [ ]:
def get_tracks_df(playlist_id: str, env_file_path: str) -> pd.DataFrame:
    tracks_data = get_tracks_data(playlist_id, env_file_path)
    tracks_audio_features = get_tracks_audio_features(playlist_id, env_file_path)
    tracks_data_and_audio_features = [(d + af) for d, af in zip(tracks_data, tracks_audio_features)]
    return pd.DataFrame(tracks_data_and_audio_features,
                        columns=[
                            'URI',
                            'Title',
                            'Popularity',
                            'Duration',
                            'Key',
                            'Mode',
                            'Tempo',
                            'time_signature',
                            'valence',
                            'danceability',
                            'energy',
                            'loudness',
                            'acousticness',
                            'instrumentalness',
                            'liveness',
                            'speechiness'
                        ])

In [ ]:
# Test get_tracks_df()
tracks_df = get_tracks_df(THE_BEATLES_PLAYLIST_URL_1, 'env/.env')
display(tracks_df)